In [1]:
from GenZ import get_model_df, get_summary_table, System, create_inference_moe_prefill_layer, create_inference_moe_decode_layer
import pandas as pd


### LLama7B Dense LLM Prefill Stage

In [2]:
TPU = System(flops=300, offchip_mem_bw=1200, compute_efficiency=0.8, memory_efficiency=0.8, bits='bf16')

In [3]:
LLM_prefill = create_inference_moe_prefill_layer(1024, "llama2_7b")
model_df = get_model_df(model=LLM_prefill, system=TPU)
model_df

,Op Type,Dimension,Bound,C/M ratio,Op Intensity,Latency (msec),Cycles,C Effcy,Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB),Throughput (Tflops),Compute time (msec),Memory time (msec),Communication time (msec),Compute cycle,Memory cycle,Communication cycle
0,GEMM,"[((1, 4096, 1024), (12288, 4096), (1, 12288, 1...",Compute,3.298535,768.0,0.429497,403726.925824,0.8,103079.215104,8.0,96.0,24.0,128.0,240.0,0.429497,0.130208,0.0,403726.925824,122395.833333,0.0
1,Logit,"((1, 32, 1024, 128), (1, 32, 1024, 128), (1, 3...",Compute,1.736071,102.4,0.035791,33643.910485,0.8,8589.934592,8.0,8.0,64.0,80.0,240.0,0.035791,0.020616,0.0,33643.910485,19379.340278,0.0
2,Attend,"((1, 32, 1024, 1024), (1, 32, 1024, 128), (1, ...",Compute,1.736071,102.4,0.035791,33643.910485,0.8,8589.934592,64.0,8.0,8.0,80.0,240.0,0.035791,0.020616,0.0,33643.910485,19379.340278,0.0
3,GEMM,"[((1, 4096, 1024), (4096, 4096), (1, 4096, 102...",Compute,2.932031,682.666667,0.143166,134575.641941,0.8,34359.738368,8.0,32.0,8.0,48.0,240.0,0.143166,0.048828,0.0,134575.641941,45898.4375,0.0
4,GEMM,"[((1, 4096, 1024), (22016, 4096), (1, 22016, 1...",Compute,3.392215,789.811659,0.769515,723344.075435,0.8,184683.593728,8.0,172.0,43.0,223.0,240.0,0.769515,0.226847,0.0,723344.075435,213236.490885,0.0
5,GEMM,"[((1, 11008, 1024), (4096, 11008), (1, 4096, 1...",Compute,3.274736,762.458874,0.384757,361672.037717,0.8,92341.796864,21.5,86.0,8.0,115.5,240.0,0.384757,0.117493,0.0,361672.037717,110443.115234,0.0


In [4]:
llama_summary = get_summary_table(model_df)
llama_summary

,MACs (MFLOP),Total Data (MB),Total Weights (MB),Unused Weights (MB),KV Cache (MB),On-chip Memory Footprint (MB),Latency (msec),Cycles,Attn Latency (msec),Linear Latency (msec),Comm Latency (msec)
0,431644.213248,674.5,386.0,0,16.0,223.0,1.798518,1.690607e+06,0.071583,1.726935,0


### Llama 7B Dense LLM Decode Stage

In [5]:
LLM_decode = create_inference_moe_decode_layer(1024, "llama2_7b")
model_df = get_model_df(model=LLM_decode, system=TPU)
model_df

,Op Type,Dimension,Bound,C/M ratio,Op Intensity,Latency (msec),Cycles,C Effcy,Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB),Throughput (Tflops),Compute time (msec),Memory time (msec),Communication time (msec),Compute cycle,Memory cycle,Communication cycle
0,GEMM,"[((1, 4096, 1), (12288, 4096), (1, 12288, 1))]",Memory,0.004295,0.999675,0.097658,91798.86712,0.8,100.663296,0.007812,96.0,0.023438,96.03125,1.03077,0.000419,0.097658,0.0,394.264576,91798.86712,0.0
1,Logit,"((1, 32, 1, 128), (1, 32, 1024, 128), (1, 32, ...",Memory,0.004292,0.991288,0.008143,7654.221853,0.8,8.388608,0.007812,8.0,0.0625,8.070312,1.030189,0.000035,0.008143,0.0,32.855381,7654.221853,0.0
2,Logit,"((1, 32, 1, 128), (1, 32, 32, 128), (1, 32, 1,...",Memory,0.004284,0.962406,0.000255,239.676899,0.8,0.262144,0.007812,0.25,0.001953,0.259766,1.028115,0.000001,0.000255,0.0,1.026731,239.676899,0.0
3,Attend,"((1, 32, 1, 1024), (1, 32, 1024, 128), (1, 32,...",Memory,0.004292,0.991288,0.008143,7654.221853,0.8,8.388608,0.0625,8.0,0.007812,8.070312,1.030189,0.000035,0.008143,0.0,32.855381,7654.221853,0.0
4,Attend,"((1, 32, 1, 32), (1, 32, 32, 128), (1, 32, 1, ...",Memory,0.004284,0.962406,0.000255,239.676899,0.8,0.262144,0.001953,0.25,0.007812,0.259766,1.028115,0.000001,0.000255,0.0,1.026731,239.676899,0.0
5,GEMM,"[((1, 4096, 1), (4096, 4096), (1, 4096, 1))]",Memory,0.004295,0.999512,0.032553,30599.954393,0.8,33.554432,0.007812,32.0,0.007812,32.015625,1.030759,0.00014,0.032553,0.0,131.421525,30599.954393,0.0
6,GEMM,"[((1, 4096, 1), (22016, 4096), (1, 22016, 1))]",Memory,0.004295,0.999711,0.174971,164472.575982,0.8,180.355072,0.007812,172.0,0.041992,172.049805,1.030772,0.000751,0.174971,0.0,706.390699,164472.575982,0.0
7,GEMM,"[((1, 11008, 1), (4096, 11008), (1, 4096, 1))]",Memory,0.004295,0.999665,0.087486,82236.537006,0.8,90.177536,0.020996,86.0,0.007812,86.028809,1.030769,0.000376,0.087486,0.0,353.195349,82236.537006,0.0


In [6]:
model_df.to_csv("llama2_7b_decode.csv", index=False)

In [7]:
llama_summary = get_summary_table(model_df)
llama_summary

,MACs (MFLOP),Total Data (MB),Total Weights (MB),Unused Weights (MB),KV Cache (MB),On-chip Memory Footprint (MB),Latency (msec),Cycles,Attn Latency (msec),Linear Latency (msec),Comm Latency (msec)
0,422.05184,402.785645,386.0,0,16.5,172.049805,0.409464,384895.732005,0.016796,0.392668,0


### Mixtral 8x7B MoE LLM Prefill Stage

In [8]:
GH200 = System(flops=2000, offchip_mem_bw=4900, compute_efficiency=0.8, memory_efficiency=0.8, bits='bf16',
            off_chip_mem_size=144)

In [9]:
LLM_prefill = create_inference_moe_prefill_layer(1024, "mixtral_8x7b")
model_df = get_model_df(model=LLM_prefill, system=GH200)
model_df

,Op Type,Dimension,Bound,C/M ratio,Op Intensity,Latency (msec),Cycles,C Effcy,Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB),Throughput (Tflops),Compute time (msec),Memory time (msec),Communication time (msec),Compute cycle,Memory cycle,Communication cycle
0,GEMM,"[((1, 4096, 1024), (6144, 4096), (1, 6144, 102...",Compute,1.901508,722.823529,0.032212,30279.519437,0.8,51539.607552,8.0,48.0,12.0,68.0,1600.0,0.032212,0.01694,0.0,30279.519437,15923.947704,0.0
1,Logit,"((1, 32, 1024, 128), (1, 8, 1024, 128), (1, 32...",Memory,0.785875,110.702703,0.006832,6421.618126,0.8,8589.934592,8.0,2.0,64.0,74.0,1257.399359,0.005369,0.006832,0.0,5046.586573,6421.618126,0.0
2,Attend,"((1, 32, 1024, 1024), (1, 8, 1024, 128), (1, 3...",Memory,0.785875,110.702703,0.006832,6421.618126,0.8,8589.934592,64.0,2.0,8.0,74.0,1257.399359,0.005369,0.006832,0.0,5046.586573,6421.618126,0.0
3,GEMM,"[((1, 4096, 1024), (4096, 4096), (1, 4096, 102...",Compute,1.795869,682.666667,0.021475,20186.346291,0.8,34359.738368,8.0,32.0,8.0,48.0,1600.0,0.021475,0.011958,0.0,20186.346291,11240.433673,0.0
4,GEMM,"[((1, 4096, 256), (229376, 4096), (1, 229376, ...",Memory,0.633171,240.688353,0.474829,446338.887117,0.8,481036.337152,2.0,1792.0,112.0,1906.0,1013.073631,0.300648,0.474829,0.0,282608.848077,446338.887117,0.0
5,GEMM,"[((1, 114688, 256), (4096, 114688), (1, 4096, ...",Memory,0.632507,240.436059,0.237663,223403.61926,0.8,240518.168576,56.0,896.0,2.0,954.0,1012.011709,0.150324,0.237663,0.0,141304.424038,223403.61926,0.0


In [10]:
llama_summary = get_summary_table(model_df)
llama_summary

,MACs (MFLOP),Total Data (MB),Total Weights (MB),Unused Weights (MB),KV Cache (MB),On-chip Memory Footprint (MB),Latency (msec),Cycles,Attn Latency (msec),Linear Latency (msec),Comm Latency (msec)
0,824633.720832,3124.0,2768.0,0,4.0,1906.0,0.779842,733051.608358,0.013663,0.766179,0


In [11]:
model_df.to_csv("mixtral_8x7b_prefill_on_GH200.csv", index=False)

### Llama 7B Dense LLM Decode Stage

In [12]:
LLM_decode = create_inference_moe_decode_layer(1024, "mixtral_8x7b")
model_df = get_model_df(model=LLM_decode, system=GH200)
model_df

,Op Type,Dimension,Bound,C/M ratio,Op Intensity,Latency (msec),Cycles,C Effcy,Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB),Throughput (Tflops),Compute time (msec),Memory time (msec),Communication time (msec),Compute cycle,Memory cycle,Communication cycle
0,GEMM,"[((1, 4096, 1), (6144, 4096), (1, 6144, 1))]",Memory,0.00263,0.999593,0.011959,11241.678748,0.8,50.331648,0.007812,48.0,0.011719,48.019531,4.208602,0.000031,0.011959,0.0,29.569843,11241.678748,0.0
1,Logit,"((1, 32, 1, 128), (1, 8, 1024, 128), (1, 32, 1...",Memory,0.010423,3.864151,0.000503,472.833672,0.8,8.388608,0.007812,2.0,0.0625,2.070312,16.676671,0.000005,0.000503,0.0,4.928307,472.833672,0.0
2,Logit,"((1, 32, 1, 128), (1, 8, 32, 128), (1, 32, 1, ...",Memory,0.010093,3.459459,0.000016,15.258519,0.8,0.262144,0.007812,0.0625,0.001953,0.072266,16.149363,0.0,0.000016,0.0,0.15401,15.258519,0.0
3,Attend,"((1, 32, 1, 1024), (1, 8, 1024, 128), (1, 32, ...",Memory,0.010423,3.864151,0.000503,472.833672,0.8,8.388608,0.0625,2.0,0.007812,2.070312,16.676671,0.000005,0.000503,0.0,4.928307,472.833672,0.0
4,Attend,"((1, 32, 1, 32), (1, 8, 32, 128), (1, 32, 1, 1...",Memory,0.010093,3.459459,0.000016,15.258519,0.8,0.262144,0.001953,0.0625,0.007812,0.072266,16.149363,0.0,0.000016,0.0,0.15401,15.258519,0.0
5,GEMM,"[((1, 4096, 1), (4096, 4096), (1, 4096, 1))]",Memory,0.00263,0.999512,0.007973,7494.618509,0.8,33.554432,0.007812,32.0,0.007812,32.015625,4.208509,0.000021,0.007973,0.0,19.713229,7494.618509,0.0
6,GEMM,"[((1, 4096, 1), (57344, 4096), (1, 57344, 1))]",Memory,0.00263,0.999738,0.111615,104918.184735,0.8,469.762048,0.007812,448.0,0.109375,448.117188,4.208768,0.000294,0.111615,0.0,275.985203,104918.184735,0.0
7,GEMM,"[((1, 4096, 0), (172032, 4096), (1, 172032, 0))]",Collective,0,0.0,0.0,0.0,0.8,0.0,0.0,1344.0,0.0,1344.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,GEMM,"[((1, 28672, 1), (4096, 28672), (1, 4096, 1))]",Memory,0.00263,0.999721,0.055808,52459.341382,0.8,234.881024,0.054688,224.0,0.007812,224.0625,4.208748,0.000147,0.055808,0.0,137.992602,52459.341382,0.0
9,GEMM,"[((1, 86016, 0), (4096, 86016), (1, 4096, 0))]",Collective,0,0.0,0.0,0.0,0.8,0.0,0.0,672.0,0.0,672.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
model_df.to_csv("mixtral_8x7b_decode_on_GH200.csv", index=False)

In [14]:
llama_summary = get_summary_table(model_df)
llama_summary

,MACs (MFLOP),Total Data (MB),Total Weights (MB),Unused Weights (MB),KV Cache (MB),On-chip Memory Footprint (MB),Latency (msec),Cycles,Attn Latency (msec),Linear Latency (msec),Comm Latency (msec)
0,805.830656,2772.5,2768.0,2016.0,4.125,1344.0,0.188394,177090.007756,0.001038,0.187355,0
